# CNNの説明

DNNと比較すると、「畳み込み層（Convolutionレイヤ）」と「プーリング層（Poolingレイヤ）」が新たに加わる。  
また、出力に近い層ではDNNと同様に全結合層が用いられる。一方、入力層では、「畳み込み層」+「ReLU」+「プーリング層」などの流れになることが大きな違い。  
「畳み込み層」では入出力データに当たる「特徴マップ」に、重みに対応するフィルターを適用して「畳み込み演算」を施す。  
その際に、通常、パディング（padding）という処理を施し、元の配列のまわりをある値で埋めてから演算を施す。  
また、プーリング層とは、特徴マップの情報を圧縮する役割を持つ層こと。
畳み込み層での演算により、画像データの三次元データの形状を維持できるため、画像認識や音声認識など至る所で使われている。 

# ChainerによるLeNetの実装

In [1]:
import numpy as np
import chainer
from chainer.backends import cuda
from chainer import Function, gradient_check, report, training, utils, Variable
from chainer import datasets, iterators, optimizers, serializers
from chainer import Link, Chain, ChainList
import chainer.functions as F
import chainer.links as L
from chainer.training import extensions

In [2]:
import argparse
from chainer import Link, Chain, ChainList
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions

# Network definition
class LeNet(chainer.Chain):

    def __init__(self, class_num=10):
        super(LeNet, self).__init__()
        with self.init_scope():
            # the size of the inputs to each layer will be inferred
            self.conv1 = L.Convolution2D(None, 6, 5, stride=1)
            self.conv2 = L.Convolution2D(None, 16, 5,stride=1)
            self.fc3 = L.Linear(None, 120)
            self.fc4 = L.Linear(None, 84)
            self.fc5 = L.Linear(None, class_num)

    def __call__(self, x):
        h = F.max_pooling_2d(F.relu(self.conv1(x)), ksize=2, stride=2)
        h = F.max_pooling_2d(F.relu(self.conv2(pool1)), ksize=2, stride=2)
        h = F.relu(self.fc3(h))
        h = F.relu(self.fc4(h))    
        h = self.fc5(h) 

        return F.softmax_cross_entropy(h, t), F.accuracy(h, t)

# スクラッチで構築したCNN(LeNet)ネットワークで学習（データセットはmnist）

In [91]:
# import sys
sys.path.append('..')
import numpy as np
import keras
from keras.datasets import mnist
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import random

max_epoch =50
batch_size =16
learning_rate = 0.001

(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = x_train.reshape(60000,1,28,28) # 2次元配列を1次元に変換
x_test = x_test.reshape(10000, 1,28,28)

In [93]:
x_train.shape

(60000, 1, 28, 28)

In [94]:
# 0〜1の範囲に正規化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# データ数1/10で学習
x_train, t_train = x_train[:6000], t_train[:6000]
x_test, t_test = x_test[:1000], t_test[:1000]
num_classes = 10
t_train = keras.utils.to_categorical(t_train, num_classes)
t_test = keras.utils.to_categorical(t_test, num_classes)

In [95]:
from lenet import lenet
from layers import *
from optimizer import *
from utils import *
# モデルとoptimizerの生成
model = LeNet(input_dim =(1,28,28))
optimizer = SGD(lr=learning_rate)

data_size = len(x_train)
max_iter = data_size // batch_size
total_loss = 0
count =0
loss_list =[]
train_acc_list = []
test_acc_list = []


for epoch in range(max_epoch):
    idx = np.random.permutation(data_size)
    x_train = x_train[idx]
    t_train = t_train[idx]
    
    for iters in range(max_iter):
        xx =x_train[iters*batch_size:(iters+1)*batch_size]
        tt = t_train[iters*batch_size:(iters+1)*batch_size]
       
        # 勾配を求め、パラメータを更新
        loss = model.forward(xx,tt)
        model.backward()
        optimizer.update(model.params,model.grads)

        total_loss +=loss
        count +=1
        model.accuracy(xx,tt)
    
        if (iters +1)%max_iter ==0:
            avg_loss = total_loss/count
            train_acc = model.accuracy(x_train, t_train)
            test_acc = model.accuracy(x_test, t_test)
            print('| epoch %d | iter %d / %d | loss %.4f | train_acc %.4f | test_acc %.4f' 
                      %(epoch + 1, iters + 1, max_iter, avg_loss, train_acc,test_acc))
            loss_list.append(avg_loss)
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            total_loss, count = 0, 0
print('max_test_acc :',max(test_acc_list))           

| epoch 1 | iter 375 / 375 | loss 2.3025 | train_acc 0.1543 | test_acc 0.1730
| epoch 2 | iter 375 / 375 | loss 2.3023 | train_acc 0.1118 | test_acc 0.1260
| epoch 3 | iter 375 / 375 | loss 2.3021 | train_acc 0.1118 | test_acc 0.1260
| epoch 4 | iter 375 / 375 | loss 2.3019 | train_acc 0.1118 | test_acc 0.1260
| epoch 5 | iter 375 / 375 | loss 2.3016 | train_acc 0.1118 | test_acc 0.1260
| epoch 6 | iter 375 / 375 | loss 2.3014 | train_acc 0.1118 | test_acc 0.1260
| epoch 7 | iter 375 / 375 | loss 2.3012 | train_acc 0.1118 | test_acc 0.1260
| epoch 8 | iter 375 / 375 | loss 2.3008 | train_acc 0.1118 | test_acc 0.1260
| epoch 9 | iter 375 / 375 | loss 2.3004 | train_acc 0.1162 | test_acc 0.1270
| epoch 10 | iter 375 / 375 | loss 2.2997 | train_acc 0.1582 | test_acc 0.1560
| epoch 11 | iter 375 / 375 | loss 2.2983 | train_acc 0.2090 | test_acc 0.2110
| epoch 12 | iter 375 / 375 | loss 2.2946 | train_acc 0.3365 | test_acc 0.3310
| epoch 13 | iter 375 / 375 | loss 2.2773 | train_acc 0.1712 

 ## データ数を1/10に絞り学習した結果、検証データのaccuracyが約97%
 

# mnistをAlexNet (keras)で学習

In [296]:
import sys
sys.path.append('..')
import numpy as np
import keras
from keras.datasets import mnist
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import random

learning_rate = 0.001

(x_train, t_train), (x_test, t_test) = mnist.load_data()

x_train = x_train.reshape(60000,1,28,28) # 2次元配列を1次元に変換
x_test = x_test.reshape(10000, 1,28,28)

In [297]:
# 0〜1の範囲に正規化
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# データ数1/10で学習
x_train, t_train = x_train[:6000], t_train[:6000]
x_test, t_test = x_test[:1000], t_test[:1000]
num_classes = 10
t_train = keras.utils.to_categorical(t_train, num_classes)
t_test = keras.utils.to_categorical(t_test, num_classes)

In [299]:
from keras.models import  Sequential

from keras.layers import *

model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11),
                 strides=(4, 4),
                 padding='same',
                 activation='relu',
                 input_shape=(1,28,28)))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(5, 5),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(384, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.SGD(),
              metrics=['accuracy'])

In [300]:
model.fit(x_train, t_train,
          epochs=50,
          batch_size=16)
score = model.evaluate(x_test, t_test,batch_size=32)
print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))

Epoch 1/50
6000/6000 [==============================] - 55s 9ms/step - loss: 0.6766 - acc: 0.7962
Epoch 2/50
6000/6000 [==============================] - 53s 9ms/step - loss: 0.2529 - acc: 0.9245
Epoch 3/50
6000/6000 [==============================] - 54s 9ms/step - loss: 0.1569 - acc: 0.9538
Epoch 4/50
6000/6000 [==============================] - 54s 9ms/step - loss: 0.1147 - acc: 0.9658
Epoch 5/50
6000/6000 [==============================] - 53s 9ms/step - loss: 0.1088 - acc: 0.9660
Epoch 6/50
6000/6000 [==============================] - 54s 9ms/step - loss: 0.0675 - acc: 0.9788
Epoch 7/50
6000/6000 [==============================] - 53s 9ms/step - loss: 0.0740 - acc: 0.9775
Epoch 8/50
6000/6000 [==============================] - 56s 9ms/step - loss: 0.0609 - acc: 0.9787
Epoch 9/50
6000/6000 [==============================] - 58s 10ms/step - loss: 0.0559 - acc: 0.9820
Epoch 10/50
6000/6000 [==============================] - 58s 10ms/step - loss: 0.0382 - acc: 0.9885
Epoch 11/50
6000/

## testデータで検証した結果、accuracyは95.4%

# スクラッチで構築したAlexNetで学習（データセットはcifar10）

# ミニバッチ学習

In [357]:
import sys
sys.path.append('..')
import numpy as np
import keras
from keras.datasets import cifar10
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import random

max_epoch = 50
batch_size =32
learning_rate = 0.001

(x_train, t_train), (x_test, t_test) = cifar10.load_data()

In [358]:
x_train.shape,x_test.shape

((50000, 32, 32, 3), (10000, 32, 32, 3))

In [359]:
# 0〜1の範囲に正規化
x_train = x_train.transpose(0,3,1,2)
x_test = x_test.transpose(0,3,1,2)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
# データ数1/10で学習
x_train, t_train = x_train[:1000], t_train[:1000]
x_test, t_test = x_test[:200], t_test[:200]

num_classes = 10
t_train = keras.utils.to_categorical(t_train, num_classes)
t_test = keras.utils.to_categorical(t_test, num_classes)

In [360]:
x_train.shape

(1000, 3, 32, 32)

In [361]:
from lenet import lenet
from layers import *
from optimizer import *
from utils import *

# モデルとoptimizerの生成
model = AlexNet(input_dim =(3,32,32))
optimizer = Adam(lr=learning_rate)

data_size = len(x_train)
max_iter = data_size // batch_size
total_loss = 0
count =0
loss_list =[]
train_acc_list = []
test_acc_list = []


for epoch in range(max_epoch):
    idx = np.random.permutation(data_size)
    x_train = x_train[idx]
    t_train = t_train[idx]
    
    for iters in range(max_iter):
        xx =x_train[iters*batch_size:(iters+1)*batch_size]
        tt = t_train[iters*batch_size:(iters+1)*batch_size]
       
        # 勾配を求め、パラメータを更新
        loss = model.forward(xx,tt)
        model.backward()
        optimizer.update(model.params,model.grads)

        total_loss +=loss
        count +=1
        #model.accuracy(xx,tt)
    
        if (iters +1)%max_iter ==0:
            avg_loss = total_loss/count
            train_acc = model.accuracy(x_train, t_train)
            test_acc = model.accuracy(x_test, t_test)
            print('| epoch %d | iter %d / %d | loss %.4f | train_acc %.4f | test_acc %.4f' 
                      %(epoch + 1, iters + 1, max_iter, avg_loss, train_acc,test_acc))
            loss_list.append(avg_loss)
            train_acc_list.append(train_acc)
            test_acc_list.append(test_acc)
            total_loss, count = 0, 0
print('max_test_acc :',max(test_acc_list))           

| epoch 1 | iter 31 / 31 | loss 10.4867 | train_acc 0.2230 | test_acc 0.1750
| epoch 2 | iter 31 / 31 | loss 6.8676 | train_acc 0.2260 | test_acc 0.1700
| epoch 3 | iter 31 / 31 | loss 3.5849 | train_acc 0.2500 | test_acc 0.1350
| epoch 4 | iter 31 / 31 | loss 2.5800 | train_acc 0.2500 | test_acc 0.1600
| epoch 5 | iter 31 / 31 | loss 2.2237 | train_acc 0.2980 | test_acc 0.2100
| epoch 6 | iter 31 / 31 | loss 2.0598 | train_acc 0.3410 | test_acc 0.2400
| epoch 7 | iter 31 / 31 | loss 1.9365 | train_acc 0.3480 | test_acc 0.2350
| epoch 8 | iter 31 / 31 | loss 1.8783 | train_acc 0.3670 | test_acc 0.2550
| epoch 9 | iter 31 / 31 | loss 1.8627 | train_acc 0.3880 | test_acc 0.2650
| epoch 10 | iter 31 / 31 | loss 1.8268 | train_acc 0.3800 | test_acc 0.2500
| epoch 11 | iter 31 / 31 | loss 1.7637 | train_acc 0.4000 | test_acc 0.2400
| epoch 12 | iter 31 / 31 | loss 1.7817 | train_acc 0.4430 | test_acc 0.2300
| epoch 13 | iter 31 / 31 | loss 1.7273 | train_acc 0.4430 | test_acc 0.3050
| epoch

 ### データ数を1/50に絞り学習した結果、検証データのaccuracy37%で訓練データに過学習している。

# keras-AlexNetを実装し、cifar10で学習

In [1]:
import sys
sys.path.append('..')
import numpy as np
import keras
from keras.datasets import cifar10
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline
import random

(x_train, t_train), (x_test, t_test) =cifar10.load_data()

Using TensorFlow backend.


In [2]:
# 0〜1の範囲に正規化
x_train = x_train.transpose(0,3,1,2)
x_test = x_test.transpose(0,3,1,2)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
# x_train /= 255
# x_test /= 255
# データ数1/10で学習
x_train, t_train = x_train[:1000], t_train[:1000]
x_test, t_test = x_test[:200], t_test[:200]

num_classes = 10
t_train = keras.utils.to_categorical(t_train, num_classes)
t_test = keras.utils.to_categorical(t_test, num_classes)

In [3]:
from keras.models import  Sequential

from keras.layers import *

model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11),
                 strides=(4, 4),
                 padding='same',
                 activation='relu',
                 input_shape=(3,32,32)))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(256, kernel_size=(5, 5),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(384, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(Conv2D(256, kernel_size=(3, 3),
                 strides=(1, 1),
                 padding='same',
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3),
                      strides=(2,2),
                      padding='same'))
model.add(BatchNormalization())


model.add(Flatten())
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2048, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [4]:
model.fit(x_train, t_train,
          epochs=50,
          batch_size=32)

Epoch 1/50
1000/1000 [==============================] - 10s 10ms/step - loss: 2.7790 - acc: 0.1410
Epoch 2/50
1000/1000 [==============================] - 10s 10ms/step - loss: 2.3587 - acc: 0.1960
Epoch 3/50
1000/1000 [==============================] - 10s 10ms/step - loss: 2.2160 - acc: 0.1910
Epoch 4/50
1000/1000 [==============================] - 10s 10ms/step - loss: 2.1433 - acc: 0.2100
Epoch 5/50
1000/1000 [==============================] - 10s 10ms/step - loss: 2.0883 - acc: 0.2190
Epoch 6/50
1000/1000 [==============================] - 9s 9ms/step - loss: 2.0435 - acc: 0.2330
Epoch 7/50
1000/1000 [==============================] - 9s 9ms/step - loss: 2.0124 - acc: 0.2370
Epoch 8/50
1000/1000 [==============================] - 9s 9ms/step - loss: 1.9689 - acc: 0.2490
Epoch 9/50
1000/1000 [==============================] - 10s 10ms/step - loss: 1.9423 - acc: 0.2660
Epoch 10/50
1000/1000 [==============================] - 10s 10ms/step - loss: 1.9098 - acc: 0.3040
Epoch 11/50
100

In [5]:
# テストデータで検証
score = model.evaluate(x_test, t_test,batch_size=32)
print("%s: %.2f%%" % (model.metrics_names[1], score[1] * 100))

200/200 [==============================] - 0s 987us/step
acc: 31.00%


 ### 訓練データでaccuracy81%、検証データではaccuracy31%で、スクラッチ版同様、訓練データに過学習した結果になった。